In [1]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import urllib.request, json

In [19]:
url_endpoint = 'https://data.cityofchicago.org/resource/cwig-ma7x.json'
response = requests.get(url_endpoint)
data = response.json()
df = pd.DataFrame(data)

In [20]:
df

,:@computed_region_43wa_7qmu,:@computed_region_6mkv_f3dw,:@computed_region_awaf_s7ux,:@computed_region_bdys_3d7i,:@computed_region_vrxf_vc4k,address,aka_name,city,dba_name,facility_type,...,inspection_type,latitude,license_,location,longitude,results,risk,state,violations,zip
0,2,21867,44,111,64,6201 S Fairfield (2732W),FAIRFIELD ACADEMY,CHICAGO,FAIRFIELD ACADEMY,School,...,Canvass Re-Inspection,41.78086789054542,26701,"{'type': 'Point', 'coordinates': [-87.69207758...",-87.69207758441868,Pass,Risk 1 (High),IL,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...,60629
1,26,14917,48,97,29,518 W HARRISON ST,HAROLD'S SHRIMP & CHICKEN 2,CHICAGO,HAROLD'S SHRIMP & CHICKEN 2,Restaurant,...,License,41.87450605062665,2578273,"{'type': 'Point', 'coordinates': [-87.64038050...",-87.64038050243174,Pass,Risk 2 (Medium),IL,"35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTR...",60607
2,48,14920,26,676,58,858 W 33RD ST,VERONICA OCELOTL,CHICAGO,VERONICA OCELOTL,NaN,...,License,41.834908185459504,2575468,"{'type': 'Point', 'coordinates': [-87.64855875...",-87.64855875568807,Not Ready,All,IL,NaN,60608
3,9,14913,48,374,34,1255 S STATE ST,KIDS + COMPANY,CHICAGO,KIDS + COMPANY,Daycare Above and Under 2 Years,...,License,41.86593147276504,2314796,"{'type': 'Point', 'coordinates': [-87.62724010...",-87.62724010325859,Pass,Risk 1 (High),IL,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...,60605
4,8,22248,49,698,54,4888 S ARCHER AVE,BIG QUESADILLAS MACHETES,CHICAGO,MACHETES PA MATAR EL HAMBRE,Restaurant,...,Short Form Complaint,41.804551467287304,2354419,"{'type': 'Point', 'coordinates': [-87.72002031...",-87.72002031513671,Pass,Risk 1 (High),IL,"34. FLOORS: CONSTRUCTED PER CODE, CLEANED, GOO...",60632
5,8,21867,49,801,61,5534 S St Louis (3500W),SANDOVAL,CHICAGO,SANDOVAL,School,...,Canvass Re-Inspection,41.792351773241066,26721,"{'type': 'Point', 'coordinates': [-87.71102416...",-87.71102416916331,Pass,Risk 1 (High),IL,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...,60629
6,31,21867,44,293,64,6536 S KEDZIE AVE,DUNKIN DONUTS,CHICAGO,DUNKIN DONUTS,Restaurant,...,License,41.774244210348684,2535470,"{'type': 'Point', 'coordinates': [-87.70317482...",-87.70317482230865,Pass,Risk 2 (Medium),IL,38. VENTILATION: ROOMS AND EQUIPMENT VENTED AS...,60629
7,42,22212,33,553,71,10618 S WESTERN AVE,BARNEY CALLAGHAN'S PUB,CHICAGO,BARNEY CALLAGHAN'S PUB,Liquor,...,License,41.70030165449612,2569749,"{'type': 'Point', 'coordinates': [-87.68157331...",-87.68157331460962,Fail,Risk 3 (Low),IL,"11. ADEQUATE NUMBER, CONVENIENT, ACCESSIBLE, D...",60643
8,31,21867,44,293,64,6500 S KEDZIE AVE,DUNKIN DONUTS,CHICAGO,DUNKIN DONUTS,Restaurant,...,Canvass,41.77527445745754,1621292,"{'type': 'Point', 'coordinates': [-87.70319733...",-87.70319733305737,Out of Business,Risk 2 (Medium),IL,NaN,60629
9,30,4300,6,198,69,3300 W 87TH ST,CHICKEN COOP,CHICAGO,CHICKEN COOP,Restaurant,...,License,41.7352585267498,2578270,"{'type': 'Point', 'coordinates': [-87.70463151...",-87.70463151056376,Pass,Risk 2 (Medium),IL,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...,60652
